# Building Name Extraction

In [1]:
# imports 
import data_preprocessor
from config import *
import pandas as pd
import data_processor
import re

In [2]:
df = data_preprocessor.load_corpus(fname, pandas = True, header = True)
# df = df.drop(columns=['Title', 'Created', 'Close Time', 'Queue'], axis=1) 
df = df.drop(columns=['Building Name'], axis=1) 

df_normalized = data_preprocessor.load_corpus(fname_normalized, pandas = True, header = True)
df_normalized = df_normalized.fillna('None') # -> if from file

print('Dataframes Loaded')

Dataframes Loaded


In [4]:
# import building_extraction

# def building_name_extraction_pipeline():
#     lst_addresses, lst_building_names = building_extraction.preparing_extraction(df, df_normalized)
    # updated_lst_addresses = data_processor.brew_address_list(lst_addresses)
#     # refine buildings name list (eg: lst_building_names.remove('nehr - e - khayyam'), which ever of these removed, add in address list)
#     extracted_building_names = building_extraction.extract_building_names(updated_lst_addresses, lst_building_names)
    
    
def building_name_extraction_pipeline():
    pass

In [4]:
# lst_addresses, lst_building_names = building_extraction.preparing_extraction(df, df_normalized)

# print(lst_addresses)
# print(lst_building_names)

# lst_building_names.remove('nehr - e - khayyam')

['House # B6, Block-B Floor Aftab Sultan residencial Complex, Shahra e Ghalib, Block 2, Clifton, Karachi', 'House # Aftab Sultan Resedention complex Appt A/3 2nd Fl, Khayaban-e-Bedi, Block 2, Clifton, Karachi', 'House # St 20 fL B2 Aftab sultan near postoffice, Khayaban-e-Bedi, Block 2, Clifton, Karachi', 'House # Aftab Sultan Complex Appt A6, Khayaban-e-Bedi, Block 2, Clifton, Karachi', 'House # ST-20 Flat B-4 2nd Floor Aftab Sultan Residential, Khayaban-e-Bedi, Block 2, Clifton, Karachi', 'House # C-7 3rd Floor Al Habib Castle, 3rd Avenue, Block 5, Clifton, Karachi', 'House # Blessed Home, Flat E-5, McNeil Road, Civil Lines, Clifton, Karachi', 'House # 37 Flat F 2 1st Floor Blessed Homes, Mc Neil Road, Civil Lines, Clifton, Karachi', 'House # Blessed Homes Appt # E-6 5TH Floor, McNeil Road, Civil Lines, Clifton, Karachi', 'House # 37-A Flat D-4 3rd Floor Blessed Homes, McNeil Road, Civil Lines, Clifton, Karachi', 'House # 37 A Apartment No C 7 F Blessed Homes, Adam Road, Civil Lines,

In [9]:
# identify field in which the address is wrongly placed
# change spelling in address if incorrect
# remove/delete building name from that field and place in building name column 
# if None not in building name:
    # check based on probability identifier mechanim
    # either place in buildng number colum or area/subarea column or other relevant place
# if No building name found then go to city building corpus and check in that
# correct other fields if they are misplaced as well.
def correction():
    pass


# when all previous done, have city building corpus ready
# check in it it building is present, if not then place new buildings in the corpus
# that way we can have a city wise building corpus/directory which is self learning and self updating
# the more runs and more it learns and saves and better results acheived
def updating_building_corpus():
    pass 

# all if building is found



In [3]:
import pandas as pd

# Sample DataFrame
data = {
    'Ticket Number': [1, 2, 3, 4, 5],
    'Address': ['Addr1', 'Addr2', 'Addr3', 'Addr4', 'Addr5'],
    'Building Name': [None, 'Building A', 'Building B', None, 'Building C'],
    'Type': ['house', 'apartment', 'house', 'apartment', 'apartment']
}

df = pd.DataFrame(data)

# 1. Filter out addresses where Building Name is None
address_list = df[df['Building Name'].isna()]['Address'].tolist()

# 2. Filter out building names which are not None and type is apartment
building_names_df = df[(df['Building Name'].notna()) & (df['Type'] == 'apartment')]

# 3. Refine the building names
# For the sake of demonstration, let's say the refinement process is a function that sets 'Building C' to None
def refine_building_name(name):
    if name == 'Building C':
        return None
    return name

# building_names_df['Building Name'] = building_names_df['Building Name'].apply(refine_building_name)
building_names_df.loc[:, 'Building Name'] = building_names_df['Building Name'].apply(refine_building_name)

# Detect which building names were set to None during refinement
removed_building_addresses = building_names_df[building_names_df['Building Name'].isna()]['Address'].tolist()

# 4. Add the addresses corresponding to the removed building names to the address list
address_list.extend(removed_building_addresses)

print(address_list)
print(removed_building_addresses)


['Addr1', 'Addr4', 'Addr5']
['Addr5']


In [ ]:
def refine_building_names(building_name):
    if not re.match(r'^(building( all blocks| block)?|block)( [a-zA-Z0-9]+)?$', building_name, re.I):
        return building_name
    return 'None'


def preparing_extraction(address_df, normalized_df):
    normalized_df = normalized_df.merge(address_df, left_on='Ticket #', right_on='Ticket#', how='left').drop('Ticket#', axis=1)
    
    # lst_addresses = normalized_df[normalized_df['Building Name'] == 'None']['Address'].tolist()
    building_names_df = normalized_df[(normalized_df['Building Name'] != 'None') & (normalized_df['Type'] == 'apartment')]
    
    building_names_df.loc[:, 'Building Name'] = building_names_df['Building Name'].apply(refine_building_name)
    
    # cleaned_building_names = building_names_df[building_names_df['Building Name'] != 'None']['Building Name'].tolist()
    cleaned_building_names = list(set(building_names_df[(building_names_df['Building Name'] != 'None') & (building_names_df['Type'] == 'apartment')]['Building Name']))
    cleaned_building_names = [building_name.lower() for building_name in cleaned_building_names]
    
    # removed_building_addresses = building_names_df[building_names_df['Building Name'] == 'None']['Address'].tolist()
    # lst_addresses.extend(removed_building_addresses)
    # return lst_addresses, cleaned_building_names

    lst_addresses = building_names_df[building_names_df['Building Name'] == 'None']['Address'].tolist()
    
    updated_lst_addresses = data_processor.brew_address_list(lst_addresses)
    
    return updated_lst_addresses, cleaned_building_names




In [10]:
from fuzzywuzzy import process

def extract_building_names(addresses, building_names, threshold=75):
    extracted_names = []

    for address in addresses:
        best_match, score = process.extractOne(address, building_names)

        if score >= threshold:
            extracted_names.append(best_match)
        else:
            extracted_names.append('None')

    return extracted_names


# print(lst_addresses)
# ebns = extract_building_names(lst_addresses, lst_building_names)

ebns = extract_building_names(updated_lst_addresses, lst_building_names)
print(ebns)

['aftab sultan residential complex', 'aftab sultan residential complex', 'aftab sultan residential complex', 'aftab sultan residential complex', 'aftab sultan residential complex', 'al habib castle apartment', 'None', 'florida homes', 'florida homes', 'florida homes', '70riviera luxury apartment', 'florida homes', 'florida homes', 'florida homes', 'florida homes', 'None', 'florida homes', 'ferrer town', 'florida homes', 'None', 'florida homes', 'florida homes', 'ferrer town', 'crystal hills apartment', 'emerald tower', 'florida homes', 'florida homes', 'ocean mall', 'florida homes', 'florida homes', 'florida homes', 'parsa heaven', 'ferrer town', 'prince complex', 'quality heights', 'royal homes', 'royal homes', 'royal homes', 'royal homes', 'royal homes', 'belle view apartment', 'belle view apartment', 'sea view apartments', 'belle view apartment', 'sea world residency', 'sky tower', 'sky tower', 'sky tower', 'sky tower', 'sky tower', 'sky tower', 'sky tower', 'sea breeze centre', 'al

In [9]:
data = {
    'Addresses':lst_addresses,
    'Extracted Building Name':ebns
}

df = pd.DataFrame(data)

df.to_excel('data/buildings/extraction_2.xlsx', sheet_name = 'Sheet1', index=False)

In [ ]:
''' results ranking: 
    extraction 2: best
    extraction: better
    extraction 3: worst'''
 
# use extraction 2 method and place all in pipeline 
# integration in normalization through ticket #

In [4]:
''' 
make a funtion which does the following:
- checks if a file name of the format {city_name}_buildings.txt exist
- if it does then calls the load_corpus function to load data from it into a list called buildings_corpus_lst
- if not then directly initializes a list called buildings_corpus_lst from the 'Building Name' column of a dataframe
- initializes a list called buildings_lst from the 'Building Name' column of a dataframe 
- make a manual checking mechaninm which asks the user to delete or edit building names in the buildings_lst
- after the user has approved the building list the program will append building names from buildings_lst to buildings_corpus_lst only if the building name is not already in buildings_corpus_lst
- this way there will be no repeatation in buildings_corpus_lst and they will be set as our standart. 
'''

import os
import pandas as pd
import data_preprocessor

# def load_corpus(file_name):
#     with open(file_name, 'r') as file:
#         return file.readlines()

def manage_building_names(df, city_name):
    file_name = f"data/buildings/directory/{city_name}_buildings.txt"

    # Check if file exists
    if os.path.exists(file_name):
        buildings_corpus_lst = data_preprocessor.load_corpus(file_name)
    else:
        buildings_corpus_lst = []

    buildings_lst = df['Building Name'].tolist()

    # Manual checking mechanism
    print("Enter 'd' to delete, 'e' to edit, or any other key to continue.")
    for index, building in enumerate(buildings_lst):
        print(f"Building Name: {building}")
        action = input("Press key: ").lower()

        if action == 'd':
            buildings_lst.pop(index)
        elif action == 'e':
            new_name = input("Enter new building name: ").lower()
            buildings_lst[index] = new_name

    # Append unique building names from buildings_lst to buildings_corpus_lst
    for building in buildings_lst:
        if building not in buildings_corpus_lst:
            buildings_corpus_lst.append(building)

    # If you want to save the updated buildings_corpus_lst back to the file
    # with open(file_name, 'w') as file:
    #     file.writelines(buildings_corpus_lst)

    return buildings_corpus_lst

# Sample usage:
df = pd.DataFrame({'Building Name': ['Building A', 'Building B', 'Building C']})
updated_buildings_corpus_lst = manage_building_names(df, "lahore")
print(updated_buildings_corpus_lst)


Building Name: Building A
Building Name: Building B
Building Name: Building C
['hello', 'Building B', 'Building C']


In [ ]:
def preparing_extraction(address_df, normalized_df):
    
    normalized_df = normalized_df.merge(address_df, left_on='Ticket #', right_on='Ticket#', how='left').drop('Ticket#', axis=1)
    
    
    city_name = list(set(normalized_df['City']))[0]    
    file_name = f"data/buildings/directory/{city_name}_buildings.txt"

    if os.path.exists(file_name):
        buildings_corpus_lst = data_preprocessor.load_corpus(file_name)
    else:
        buildings_corpus_lst = []


    building_names_df = normalized_df[(normalized_df['Building Name'] != 'None') & (normalized_df['Type'] == 'apartment')]
    building_names_df.loc[:, 'Building Name'] = building_names_df['Building Name'].apply(refine_building_name)
    
    buildings_lst = list(set(building_names_df[(building_names_df['Building Name'] != 'None') & (building_names_df['Type'] == 'apartment')]['Building Name']))
    buildings_lst = [building_name.lower() for building_name in cleaned_building_names]


    print("Commands: 'd' - delete, 'e' - edit, any other key - continue.")
    for index, building in enumerate(buildings_lst):
        print(f"Building Name: {building}")
        action = input("Enter Command: ").lower()
        if action == 'd':
            buildings_lst.pop(index)
        elif action == 'e':
            new_name = input("Enter new Building Name: ").lower()
            buildings_lst[index] = new_name

    for building in buildings_lst:
        if building not in buildings_corpus_lst:
            buildings_corpus_lst.append(building)
            
    datapreprocessor.create_corpus(file_name, buildings_corpus_lst)
 
    
    lst_addresses = building_names_df[building_names_df['Building Name'] == 'None']['Address'].tolist()
    updated_lst_addresses = data_processor.brew_address_list(lst_addresses)
    
    return updated_lst_addresses, buildings_corpus_lst





In [ ]:
def preparing_extraction(address_df, normalized_df):
    normalized_df = normalized_df.merge(address_df, left_on='Ticket #', right_on='Ticket#', how='left').drop('Ticket#', axis=1)
    
    building_names_df = normalized_df[(normalized_df['Building Name'] != 'None') & (normalized_df['Type'] == 'apartment')]
    building_names_df.loc[:, 'Building Name'] = building_names_df['Building Name'].apply(refine_building_name)
    
    cleaned_building_names = list(set(building_names_df[(building_names_df['Building Name'] != 'None') & (building_names_df['Type'] == 'apartment')]['Building Name']))
    cleaned_building_names = [building_name.lower() for building_name in cleaned_building_names]
    
    lst_addresses = building_names_df[building_names_df['Building Name'] == 'None']['Address'].tolist()
    updated_lst_addresses = data_processor.brew_address_list(lst_addresses)
    
    return updated_lst_addresses, cleaned_building_names